In [1]:
import requests
import sys
import pandas as pd

/Users/deeppatel/Desktop/Meditation/Thesis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Biomart

In [4]:
server = "http://rest.ensembl.org"
ext = "/lookup/id/ENSG00000289627?"
r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
if not r.ok:
  r.raise_for_status()
  sys.exit()

decoded = r.json()
print(repr(decoded))


{'canonical_transcript': 'ENST00000693039.1', 'object_type': 'Gene', 'id': 'ENSG00000289627', 'source': 'havana_tagene', 'end': 54582580, 'biotype': 'lncRNA', 'db_type': 'core', 'seq_region_name': '2', 'assembly_name': 'GRCh38', 'version': 1, 'species': 'homo_sapiens', 'description': 'novel transcript, sense intronic to SPTBN1', 'start': 54581577, 'logic_name': 'havana_tagene_homo_sapiens', 'strand': 1}


In [3]:
start = decoded['start']
end = decoded['end']
display_name = decoded['display_name']
chrom_num = decoded['seq_region_name']
bio_type = decoded['biotype']
object_type = decoded['object_type']

In [7]:
#Reading the doc
df = pd.read_csv("/Users/deeppatel/Desktop/Meditation/Thesis/DEG_T2_T3/DE_gene_TE_analysis.txt",sep='\t')

In [10]:
df = df[df['pvalue'] < 0.05]

In [23]:
df = df.iloc[0:609]

In [61]:
df_new = pd.DataFrame(columns = ['Ensembl_ID','log2FoldChange','pvalue','Object_name','chr','start','end'])

In [62]:
for i in range(len(df)):
    ext = "/lookup/id/"+str(df.index[i].split(".")[0])+"?"
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    decoded = r.json()
    if len(decoded) > 1:  
        start = decoded['start']
        end = decoded['end']
        if 'display_name' in decoded.keys():
            display_name = decoded['display_name']
        else:
            if 'biotype' in decoded.keys() and decoded['biotype'] == 'lncRNA':
                display_name = "lncRNA:"+decoded['description']
        chrom_num = decoded['seq_region_name']
        object_type = decoded['object_type']
        row = [df.index[i],df['log2FoldChange'][i],df['pvalue'][i],display_name,chrom_num,start,end]
        df_new.loc[len(df_new)] = row
    

/var/folders/f3/b6p9hb4s1796fx3xwg65bg_r0000gn/T/ipykernel_1058/2390015646.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row = [df.index[i],df['log2FoldChange'][i],df['pvalue'][i],display_name,chrom_num,start,end]
/var/folders/f3/b6p9hb4s1796fx3xwg65bg_r0000gn/T/ipykernel_1058/2390015646.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row = [df.index[i],df['log2FoldChange'][i],df['pvalue'][i],display_name,chrom_num,start,end]
/var/folders/f3/b6p9hb4s1796fx3xwg65bg_r0000gn/T/ipykernel_1058/2390015646.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, int

In [64]:
df_new.to_csv("DE_Sig_v1.csv",index=False)

## Classification of lncRNA

In [6]:
df_lnc = pd.read_csv("/Users/deeppatel/Desktop/Meditation/Thesis/DEG_T2_T3/lncipedia_5_1_hg38.bed",sep='\t',header=None)

In [8]:
lnc_list = df_lnc[3].tolist()

In [10]:
#Seperate the name of lncRNA having : in it
lnc_list_new = []
for i in lnc_list:
    if ":" in i:
        lnc_list_new.append(i.split(":")[0])
    else:
        lnc_list_new.append(i)

In [14]:
df_DEG = pd.read_csv("DE_Sig_v1.csv")

In [20]:
df_main_lnc = pd.DataFrame(columns = ['Ensembl_ID','log2FoldChange','pvalue','Object_name','chr','start','end'])
df_main_up = pd.DataFrame(columns = ['Ensembl_ID','log2FoldChange','pvalue','Object_name','chr','start','end'])
df_main_down = pd.DataFrame(columns = ['Ensembl_ID','log2FoldChange','pvalue','Object_name','chr','start','end'])

In [21]:
for i in range(len(df_DEG)):
    if df_DEG['Object_name'][i] in lnc_list_new:
        df_main_lnc.loc[len(df_main_lnc)] = df_DEG.iloc[i]
    elif 'lncRNA' in df_DEG['Object_name'][i]:
        df_main_lnc.loc[len(df_main_lnc)] = df_DEG.iloc[i]
    elif df_DEG['log2FoldChange'][i] >= 1:
        df_main_up.loc[len(df_main_up)] = df_DEG.iloc[i]
    elif df_DEG['log2FoldChange'][i] <= -1:
        df_main_down.loc[len(df_main_down)] = df_DEG.iloc[i]

In [19]:
df_main_lnc.to_csv("DEG_lncRNA.csv",index=False)

In [26]:
df_main_up.to_csv("DEG_up.csv",index=False)

In [27]:
df_main_down.to_csv("DEG_down.csv",index=False)

In [8]:
df_main_lnc = pd.read_csv("DEG_lncRNA.csv")

## TF Finding Algo

In [9]:
api = "https://api.genome.ucsc.edu/getData/track?genome=hg38;track=rmsk;chrom=chr1;start=4000;end=48000"

In [10]:
r = requests.get(api)

In [11]:
decoded = r.json()

In [31]:
decoded.keys()

dict_keys(['downloadTime', 'downloadTimeStamp', 'genome', 'trackType', 'track', 'chrom', 'chromSize', 'bigDataUrl', 'start', 'end', 'jaspar2022', 'itemsReturned'])

In [24]:
df_TE = pd.DataFrame(columns=['lncRNA','chr','lnc_start','lnc_end','Alu','strand','alu_chr','alu_start','alu_end'])
for i in range(len(df_main_lnc)):
    api = "https://api.genome.ucsc.edu/getData/track?genome=hg38;track=rmsk;chrom="+str(df_main_lnc['chr'][i])+";start="+str(df_main_lnc['start'][i])+";end="+str(df_main_lnc['end'][i])
    r = requests.get(api)
    decoded = r.json()
    if 'rmsk' not in indecoded.keys():
        continue
    elif len(decoded['rmsk']) > 0:
        for j in range(len(decoded['rmsk'])):
            if decoded['rmsk'][j]['repFamily'] == 'Alu':
                row = [df_main_lnc['Object_name'][i],df_main_lnc['chr'][i],df_main_lnc['start'][i],df_main_lnc['end'][i],decoded['rmsk'][j]['repName'],decoded['rmsk'][j]['strand'],decoded['rmsk'][j]['genoName'],decoded['rmsk'][j]['genoStart'],decoded['rmsk'][j]['genoEnd']]
                df_TE.loc[len(df_TE)] = row

In [27]:
df_TF = pd.DataFrame(columns=['lncRNA','chr','lncRNA_start','lncRNA_end','Alu','Alu_strand','Alu_chr','Alu_start','Alu_end','TF_Name','TF_MA','Score','TF_strand','TF_chr','TF_start','TF_end'])
for i in range(len(df_TE)):
    api = "https://api.genome.ucsc.edu/getData/track?genome=hg38;track=jaspar2022;chrom="+str(df_TE['chr'][i])+";start="+str(df_TE['alu_start'][i])+";end="+str(df_TE['alu_end'][i])
    r = requests.get(api)
    decoded = r.json()
    df_temp = pd.DataFrame(decoded['jaspar2022'])
    #Filter the TF with score > 400
    df_temp = df_temp[df_temp['score'] > 400]
    df_temp = df_temp.reset_index()
    for j in range(len(df_temp)):
        row = [df_TE['lncRNA'][i],df_TE['chr'][i],df_TE['lnc_start'][i],df_TE['lnc_end'][i],df_TE['Alu'][i],df_TE['strand'][i],df_TE['chr'][i],df_TE['alu_start'][i],df_TE['alu_end'][i],df_temp['name'][j],df_temp['score'][j],df_temp['score'][j],df_temp['strand'][j],df_temp['chrom'][j],df_temp['chromStart'][j],df_temp['chromEnd'][j]]
        df_TF.loc[len(df_TF)] = row
    

KeyboardInterrupt: 

In [29]:
df_TE

,lncRNA,chr,lnc_start,lnc_end,Alu,strand,alu_chr,alu_start,alu_end
0,LINC00467,1,211382736,211435570,AluJr,-,chr1,211384270,211384552
1,LINC00467,1,211382736,211435570,AluSz6,-,chr1,211385047,211385322
2,LINC00467,1,211382736,211435570,AluY,-,chr1,211387546,211387852
3,LINC00467,1,211382736,211435570,AluJo,+,chr1,211389806,211389958
4,LINC00467,1,211382736,211435570,AluSx1,+,chr1,211390791,211391088
...,...,...,...,...,...,...,...,...,...
1471,"lncRNA:novel transcript, antisense to ANK1",1,145917714,145926741,AluSc,-,chr1,145920033,145920255
1472,"lncRNA:novel transcript, antisense to ANK1",1,145917714,145926741,AluSz6,+,chr1,145921946,145922220
1473,"lncRNA:novel transcript, antisense to ANK1",1,145917714,145926741,AluY,-,chr1,145922926,145923228
1474,"lncRNA:novel transcript, antisense to ANK1",1,145917714,145926741,AluJo,+,chr1,145925405,145925622
